In [21]:
from comet import download_model, load_from_checkpoint

model_path = download_model("wmt21-comet-qe-da")
model = load_from_checkpoint(model_path)

wmt21-comet-qe-da.tar.gz:  34%|███▍      | 584M/1.72G [00:21<00:42, 27.1MB/s]
Predicting DataLoader 0:   0%|          | 0/1 [3:02:33<?, ?it/s]

In [14]:
import json
import os
with open('/export/home/cond-text-gen/outputs/opus_bfs_-1_5_5_False_0.0_0.0_5_1.0_1.0_1_output.json','r') as fd:
    data = json.load(fd)
print(data[0])


{'ref': ['Das hat der Hollywood-Star in einem Interview jetzt unmissverständlich klar gemacht.'], 'input': 'That is what the Hollywood star has made abundantly clear in an interview.', 'uid': 0, 'output': ['Und genau das hat der Hollywood-Star im Interview deutlich gemacht.', 'Und genau das hat der Hollywoodstar in einem Interview deutlich gemacht.', 'Und das hat der Hollywoodstar in einem Interview deutlich gemacht.', 'So hat der Hollywood-Star in einem Interview ausführlich deutlich gemacht.', 'So hat der Hollywood-Star in einem Interview deutlich gemacht.', 'Genau das hat der Hollywood-Star in einem Interview deutlich gemacht.', 'Das hat der Hollywood-Star in einem Interview deutlich gemacht.', 'Dies hat der Hollywood-Star im Interview deutlich gemacht.', 'Und genau das hat der Hollywoodstar im Interview deutlich gemacht.']}


In [18]:
import statistics
for d in data:
    ref = d['ref']
    assert len(ref) == 1
    outputs = d['output']
    src = d['input']
    inputs_for_comet = ref + outputs
    feed = []
    for x in inputs_for_comet:
        feed.append(
            {
                'src': src,
                'mt':x
            }
        )
    seg_scores, sys_score = model.predict(feed, batch_size=1, gpus=0,progress_bar=False)
    ref_score = seg_scores[0]
    pred_scores = seg_scores[1:]
    print(f"REF: {ref_score}")
    print(pred_scores)
    break



/root/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/root/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=8)`.
  rank_zero_warn(


REF: 0.14903849363327026
[0.14904716610908508, 0.15392330288887024, 0.1537102311849594, 0.14234232902526855, 0.14837831258773804, 0.14989137649536133, 0.15360324084758759, 0.1508725881576538, 0.15033280849456787]


In [20]:
feed = [
    {'src':"今天是一个星期日，天气晴朗。", 'mt':"Today is Sunday, and the weather is nice."},
    {'src':"今天是一个星期日，天气晴朗。", 'mt':"Today is Sunday, and the weather is crappy and nice ."},
    {'src':"今天是一个星期日，天气晴朗。", 'mt':"Today is Monday but not Tuesday, and the weather is crappy."},
    {'src':"今天是一个星期日，天气晴朗。", 'mt':"Today is Sunday, and the weather is isisis is."},
]
seg_scores, sys_score = model.predict(feed, batch_size=1, gpus=0,progress_bar=False)
print(seg_scores)

/root/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/root/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1814: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=8)`.
  rank_zero_warn(


[0.15788614749908447, 0.11913292855024338, 0.1456047147512436, 0.10244011878967285]
